In [1]:
%load_ext Cython

In [2]:
!pip install line_profiler==3.3.1
%load_ext line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
! unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

Archive:  /root/nltk_data/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  


In [7]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

In [8]:
seed_everything(42)

42

In [9]:
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
train= pd.read_csv('/content/drive/MyDrive/Disaster-Tweets-Prediction-main/train.csv')
test= pd.read_csv("/content/drive/MyDrive/Disaster-Tweets-Prediction-main/test.csv")

In [ ]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

# Preprocessing without optimization

In [11]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

def clean_sentence(sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    sentence = remove_emoji(sentence)
    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

def remove_stopwords(tokens, stopwords):
    clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
def lemmatize(tokens, lemma):
    lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens



In [12]:
# fix sentence length
def trunc_padding(sentence):
    modify_sentence = sentence.copy()
    if len(modify_sentence) >= MAX_LENGTH:
        modify_sentence = modify_sentence[:MAX_LENGTH]
    else:
        modify_sentence.extend(list(["0"] * (MAX_LENGTH - len(modify_sentence))))
    return modify_sentence

In [13]:
stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

def processing(df, stopwords, lemma):
    df['text'] = df['text'].apply(lambda sentence: sentence.lower())
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    # tokenization
    df['text'] = df['text'].apply(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].apply(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].apply(lambda x: len(x))

In [10]:
%lprun -f processing processing(train, stopwords, lemma)

<pre>
Timer unit: 1e-06 s

Total time: 10.0521 s
File: <ipython-input-9-0655b95bc61f>
Function: processing at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def processing(df, stopwords, lemma):
     5         1      29022.0  29022.0      0.3      df['text'] = df['text'].apply(lambda sentence: sentence.lower())
     6         1     435270.0 435270.0      4.3      df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
     7                                               # tokenization
     8         1      23946.0  23946.0      0.2      df['text'] = df['text'].apply(lambda sentence: sentence.split())
     9                                               # remove stopwords
    10         1     379671.0 379671.0      3.8      df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    11                                               # lemmalization 
    12         1    9130331.0 9130331.0     90.8      df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    13                                               # sentence length before padding
    14         1       9110.0   9110.0      0.1      df['length'] = df['text'].apply(lambda x: len(x))
    15                                               # fix sentence length
    16         1      35964.0  35964.0      0.4      df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    17                                               # sentence length after padding
    18         1       8771.0   8771.0      0.1      df['length_padding'] = df['text'].apply(lambda x: len(x))

In [14]:
%lprun -f processing processing(test, stopwords, lemma)

<pre>
Timer unit: 1e-06 s

Total time: 0.709026 s
File: <ipython-input-13-0655b95bc61f>
Function: processing at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def processing(df, stopwords, lemma):
     5         1       3960.0   3960.0      0.6      df['text'] = df['text'].apply(lambda sentence: sentence.lower())
     6         1      58284.0  58284.0      8.2      df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
     7                                               # tokenization
     8         1       8346.0   8346.0      1.2      df['text'] = df['text'].apply(lambda sentence: sentence.split())
     9                                               # remove stopwords
    10         1      80398.0  80398.0     11.3      df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    11                                               # lemmalization 
    12         1     544493.0 544493.0     76.8      df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    13                                               # sentence length before padding
    14         1       2856.0   2856.0      0.4      df['length'] = df['text'].apply(lambda x: len(x))
    15                                               # fix sentence length
    16         1       7977.0   7977.0      1.1      df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    17                                               # sentence length after padding
    18         1       2712.0   2712.0      0.4      df['length_padding'] = df['text'].apply(lambda x: len(x))

# Preprocessing with Cython optimization

In [18]:
%%cython 
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1
# cython: profile=True
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools
nltk.download('stopwords')
nltk.download("wordnet")
cdef seed_everything(int seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
cdef int MAX_LENGTH = 30
cdef object device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cdef clean_sentence(object sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    cdef object emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r' ', sentence)

    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

cdef remove_stopwords(list tokens, object stopwords):
    cdef list clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
cdef lemmatize(list tokens, object lemma):
    cdef list lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

cdef trunc_padding(list sentence):
    sentence = list(itertools.islice(itertools.chain(sentence, itertools.repeat('0')), 0, MAX_LENGTH)) # fix sentence length
    return sentence

cdef object stopwords = nltk.corpus.stopwords.words('english')
cdef object lemma = WordNetLemmatizer()

def processing(df: pd.DataFrame, object stopwords, object lemma):
    df['text'] = df['text'].map(lambda sentence: sentence.lower())
    df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    # tokenization 123123
    df['text'] = df['text'].map(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].map(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].map(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].map(lambda x: len(x))


In [19]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download("wordnet")
train= pd.read_csv('/content/drive/MyDrive/Disaster-Tweets-Prediction-main/train.csv')
test= pd.read_csv("/content/drive/MyDrive/Disaster-Tweets-Prediction-main/test.csv")
stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
%lprun -f processing processing(train, stopwords, lemma)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<pre>
Timer unit: 1e-06 s

Total time: 8.682 s
File: /root/.cache/ipython/cython/_cython_magic_8cec306dd1bca0d781798f8b4263af96.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1       4893.0   4893.0      0.1      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1     123447.0 123447.0      1.4      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1      12871.0  12871.0      0.1      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1     181401.0 181401.0      2.1      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83                                               # lemmalization 
    84         1    8324249.0 8324249.0     95.9      df['text'] = df['text'].map(lambda sentence: lemmatize(sentence, lemma))
    85                                               # sentence length before padding
    86         1       6568.0   6568.0      0.1      df['length'] = df['text'].map(lambda x: len(x))
    87                                               # fix sentence length
    88         1      23586.0  23586.0      0.3      df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    89                                               # sentence length after padding
    90         1       4985.0   4985.0      0.1      df['length_padding'] = df['text'].map(lambda x: len(x))

In [20]:
%reload_ext line_profiler
%lprun -f processing processing(test, stopwords, lemma)


<pre>Timer unit: 1e-06 s

Total time: 0.591486 s
File: /root/.cache/ipython/cython/_cython_magic_8cec306dd1bca0d781798f8b4263af96.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1       4764.0   4764.0      0.8      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1      47267.0  47267.0      8.0      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1       6492.0   6492.0      1.1      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1       7151.0   7151.0      1.2      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83                                               # lemmalization 
    84         1     514114.0 514114.0     86.9      df['text'] = df['text'].map(lambda sentence: lemmatize(sentence, lemma))
    85                                               # sentence length before padding
    86         1       2315.0   2315.0      0.4      df['length'] = df['text'].map(lambda x: len(x))
    87                                               # fix sentence length
    88         1       7330.0   7330.0      1.2      df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    89                                               # sentence length after padding
    90         1       2053.0   2053.0      0.3      df['length_padding'] = df['text'].map(lambda x: len(x))

# Comparison
processing training data without optimization takes:  10.0521 s <br>
processing training data with Cython optimization takes:  8.682 s <br>
The speed up is : 10.0521 / 8.682 ≈ 1.16 <br>
The relative speed up is 16 %
<br><br>
processing testing data without optimization takes:  0.709026 s<br>
processing testing data with Cython optimization takes:  0.591486 s <br>
The speed up is : 0.709026 / 0.591486 ≈ 1.19<br>
The relative speed up is 19 %

